# Analysis + Evaluation (Discriminator Network)

Here we attempt to train a neural network to distinguish between ground truth recipes, GRU-produced recipes, checklist-produced recipes, GPT-2 finetuned recipes. As a discriminator neural network we implemented an LSTM.

Accuracy of distinguishing truth from GPT-2? 0.598

Accuracy of distinguishing truth from GRU? 0.690

Conclusion: I think something went wrong along the way. Will try some other neural network instead.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model

import os
import pathlib
import pandas as pd
import numpy as np
import re

from tqdm import tqdm
tqdm.pandas()

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [ ]:
CACHE_DIR = "./drive/Shared drives/Capstone/tmp"
pathlib.Path(CACHE_DIR).mkdir(exist_ok=True)
dataset_path = os.path.join(CACHE_DIR, 'recipes.pkl')

In [ ]:
!head -n 50 "./drive/Shared drives/Capstone/tmp/text_recipes.txt"

<TITLE>
Slow Cooker Chicken and Dumplings
<INGREDIENTS>
• 4 skinless, boneless chicken breast halves
• 2 tablespoons butter
• 2 (10.75 ounce) cans condensed cream of chicken soup
• 1 onion, finely diced
• 2 (10 ounce) packages refrigerated biscuit dough, torn into pieces
<INSTRUCTIONS>
‣ Place the chicken, butter, soup, and onion in a slow cooker, and fill with enough water to cover.
‣ Cover, and cook for 5 to 6 hours on High. About 30 minutes before serving, place the torn biscuit dough in the slow cooker. Cook until the dough is no longer raw in the center.
<DONE>
<TITLE>
Awesome Slow Cooker Pot Roast
<INGREDIENTS>
• 2 (10.75 ounce) cans condensed cream of mushroom soup
• 1 (1 ounce) package dry onion soup mix
• 1 1/4 cups water
• 5 1/2 pounds pot roast
<INSTRUCTIONS>
‣ In a slow cooker, mix cream of mushroom soup, dry onion soup mix and water. Place pot roast in slow cooker and coat with soup mixture.
‣ Cook on High setting for 3 to 4 hours, or on Low setting for 8 to 9 hours.
<DONE

In [ ]:
if not os.path.exists(dataset_path):
    raise SystemExit("Run preprocess_pickle.ipynb to generate data file before continuing")
else:
    recipes = pd.read_pickle(dataset_path)

# TODO: Remove subsetting for final training
recipes = recipes[:20000]

In [ ]:
recipes

,title,ingredients,instructions
0,Slow Cooker Chicken and Dumplings,"• 4 skinless, boneless chicken breast halves\n...","‣ Place the chicken, butter, soup, and onion i..."
1,Awesome Slow Cooker Pot Roast,• 2 (10.75 ounce) cans condensed cream of mush...,"‣ In a slow cooker, mix cream of mushroom soup..."
2,Brown Sugar Meatloaf,• 1/2 cup packed brown sugar\n• 1/2 cup ketchu...,‣ Preheat oven to 350 degrees F (175 degrees C...
3,Best Chocolate Chip Cookies,"• 1 cup butter, softened\n• 1 cup white sugar\...",‣ Preheat oven to 350 degrees F (175 degrees C...
4,Homemade Mac and Cheese Casserole,• 8 ounces whole wheat rotini pasta\n• 3 cups ...,‣ Preheat oven to 350 degrees F. Line a 2-quar...
...,...,...,...
20161,Georgia's Tennessee Jam Cake,"• 1 cup butter, softened\n• 2 cups white sugar...",‣ Preheat the oven to 350 degrees F (175 degre...
20162,Poached Eggs and Asparagus,• 4 eggs\n• 1 cube chicken bouillon (optional)...,‣ Fill a saucepan half way full of water. Brin...
20163,Bistecca alla Fiorentina (Tuscan Porterhouse),"• 4 sprigs fresh rosemary, chopped\n• 1 (2 1/2...",‣ Press chopped rosemary onto both sides of po...
20164,Courtney's Three Tomato Pasta Sauce,• 1/2 pound bulk mild Italian sausage\n• 1/2 p...,‣ Cook mild and hot Italian sausage in a large...


In [ ]:
def recipe_to_str(recipe):
    # Combine components of recipe into a string
    return f"🍴 {recipe.title} 🥑 {recipe.ingredients} 🥣 {recipe.instructions}"

recipe_strings = recipes.apply(recipe_to_str, axis=1)

In [ ]:
recipe_strings[:10]

0     🍴 Slow Cooker Chicken and Dumplings 🥑 • 4 skin...
1     🍴 Awesome Slow Cooker Pot Roast 🥑 • 2 (10.75 o...
2     🍴 Brown Sugar Meatloaf 🥑 • 1/2 cup packed brow...
3     🍴 Best Chocolate Chip Cookies 🥑 • 1 cup butter...
4     🍴 Homemade Mac and Cheese Casserole 🥑 • 8 ounc...
5     🍴 Banana Banana Bread 🥑 • 2 cups all-purpose f...
7     🍴 Mom's Zucchini Bread 🥑 • 3 cups all-purpose ...
8     🍴 The Best Rolled Sugar Cookies 🥑 • 1 1/2 cups...
9     🍴 Singapore Chili Crabs 🥑 • Sauce:\n• 1/2 cup ...
10    🍴 Downeast Maine Pumpkin Bread 🥑 • 1 (15 ounce...
dtype: object

# New stuff

In [ ]:
GPT2_recipe_dir = os.path.join(CACHE_DIR, 'gpt2_105791_title_prompt_output_recipes')
GPT2_recipe_path_str = 'gpt2_recipe_'

GRU_char_dir = os.path.join(CACHE_DIR, 'rnn_char_title_prompt_output_recipes')
GRU_char_path_str = 'rnn_char_recipe_'

GRU_word_dir = os.path.join(CACHE_DIR, 'rnn_word_punct_emoji_online_title_prompt_output_recipes')
GRU_word_path_str = 'rnn_word_recipe_'

checklist_dir = os.path.join(CACHE_DIR, 'checklist_53_rnn_prompt_recipes')
checklist_path_str = 'checklist_rnn_prompt_recipe_'

In [ ]:
to_exclude = '#$&*/<=>@[\\]^_`{|}~\t'
to_tokenize = '.,:;!?"-+()%\'\n'

def separate_punct(text):
    return re.sub(r'(['+to_tokenize+'])', r' \1 ', text)

In [ ]:
def recipe_to_str(recipe):
    # Combine components of recipe into a string
    return f"🍴 {recipe.title} 🥑 {recipe.ingredients} 🥣 {recipe.instructions}"

recipe_strings = recipes.apply(recipe_to_str, axis=1).apply(lambda text: separate_punct(text.lower().replace('\n', '')).strip())

In [ ]:
recipe_strings

0        🍴 slow cooker chicken and dumplings 🥑 • 4 skin...
1        🍴 awesome slow cooker pot roast 🥑 • 2  ( 10 . ...
2        🍴 brown sugar meatloaf 🥑 • 1/2 cup packed brow...
3        🍴 best chocolate chip cookies 🥑 • 1 cup butter...
4        🍴 homemade mac and cheese casserole 🥑 • 8 ounc...
                               ...                        
20161    🍴 georgia ' s tennessee jam cake 🥑 • 1 cup but...
20162    🍴 poached eggs and asparagus 🥑 • 4 eggs• 1 cub...
20163    🍴 bistecca alla fiorentina  ( tuscan porterhou...
20164    🍴 courtney ' s three tomato pasta sauce 🥑 • 1/...
20165    🍴 tom collins 🥑 • 2 fluid ounces gin• 2 fluid ...
Length: 20000, dtype: object

In [ ]:
gpt2_recipe_strings = []

for i in tqdm(range(50)):
    file_path = os.path.join(GPT2_recipe_dir, f"{GPT2_recipe_path_str}{i}.txt")
    gpt2_recipe_string = ""
    with open(file_path, 'r') as f:
        text = f.read()
        text = '🍴 ' + text.replace('<ING>', ' 🥑 ').replace('<INS>', ' 🥣 ')
        text = separate_punct(text.lower().replace('\n', '')).strip()
        gpt2_recipe_strings.append(text)


100%|██████████| 50/50 [00:00<00:00, 969.07it/s]


In [ ]:
gpt2_df = pd.DataFrame(gpt2_recipe_strings)

In [ ]:
gpt2_df.columns = ['text']

In [ ]:
gpt2_df['label'] = 4

In [ ]:
gpt2_df

,text,label
0,🍴 slow cooker turkey and potatoes 🥑 • 2 ( 16 ...,4
1,🍴 endive & gorgonzola salad 🥑 • 3 tablespoo...,4
2,🍴 curry-spiced minestrone 🥑 • 2 tablespoons ol...,4
3,🍴 tuna dip ii 🥑 • 2 ( 5 ounce ) cans solid w...,4
4,🍴 pancetta wrapped bayou la batre shrimp with ...,4
5,🍴 tightend tip ( sangria soy tri-tip ) 🥑 • 2...,4
6,🍴 slippery fish 🥑 • 2 ( 12 fluid ounce ) can...,4
7,🍴 sweet potato carrot spiced life juice 🥑 • 2 ...,4
8,🍴 cajun shrimp and cornbread stuffing 🥑 • 1 1/...,4
9,"🍴 mixed green salad with mango , jícama and c...",4


In [ ]:
gruchar_recipe_strings = []

for i in tqdm(range(50)):
    file_path = os.path.join(GRU_char_dir, f"{GRU_char_path_str}{i}.txt")
    gruchar_recipe_string = ""
    with open(file_path, 'r') as f:
        text = f.read()
        text = '🍴 ' + text.replace('🥑', ' 🥑 ').replace('🥣', ' 🥣 ')
        text = separate_punct(text.lower().replace('\n', '')).strip()
        gruchar_recipe_strings.append(text)



100%|██████████| 50/50 [00:00<00:00, 1163.41it/s]


In [ ]:
gruchar_df = pd.DataFrame(gruchar_recipe_strings)

In [ ]:
gruchar_df.columns = ['text']
gruchar_df['label'] = 1

In [ ]:
gruchar_df

,text,label
0,🍴 johnnycakes italian sausage shortcake balls ...,1
1,🍴 zucchini with garlic and dried crushed red p...,1
2,🍴 vegetarian pizza double frozen french toast ...,1
3,🍴 summer vegetable chili grilling potato bread...,1
4,🍴 pizza stix® ) • 1/2 cup white sugar• 1/2 cup...,1
5,🍴 hash-browned spaghetti squash with spaghetti...,1
6,🍴 hot eggnog cheese ) • 1 cup all-purpose flou...,1
7,🍴 sweet cornmeal cake brazilian-style bread pi...,1
8,🍴 green slaw with champagne buttercream frosti...,1
9,🍴 tasty salmon spread with creole seasoning ; ...,1


In [ ]:
gruword_recipe_strings = []

for i in tqdm(range(50)):
    file_path = os.path.join(GRU_word_dir, f"{GRU_word_path_str}{i}.txt")
    gruword_recipe_string = ""
    with open(file_path, 'r') as f:
        text = f.read()
        text = separate_punct(text.lower().replace('\n', '')).strip()
        gruword_recipe_strings.append(text)



100%|██████████| 50/50 [00:00<00:00, 1208.81it/s]


In [ ]:
gruword_df = pd.DataFrame(gruword_recipe_strings)
gruword_df.columns = ['text']
gruword_df['label'] = 2

In [ ]:
gruword_df['text'][0]

'🍴 slow braised oxtail 🥑  • 3 cups all-purpose flour  • 2 teaspoons baking powder  • 1 2 teaspoon salt  • 1 2 teaspoon baking powder  • 4 eggs  ,  beaten  • 1 2 cup unsalted butter at room temperature  • 1 2 cup white applesauce   🥣  ‣ in a large mixing bowl  ,  toss together with sugar  ,  green onion  ,  soy sauce  ,  vinegar  ,  water  ,  1 4 teaspoon garlic powder  ,  and sugar  .  set aside  .   ‣ in a medium bowl  ,  beat egg whites with sugar  .   ‣ divide the beef mixture into the bottom of the eggplant  .  form a scant 2 large slits  ,  or 1 4 cup at a time  ,  until the liquid has been added  .  fold up or the floured loaf with the sides of a fork  ,  and sprinkle with sugar  .  before decorating  ,  and top with the remaining sour cream  .  cover and refrigerate  .'

In [ ]:
checklist_recipe_strings = []

for i in tqdm(range(50)):
    file_path = os.path.join(checklist_dir, f"{checklist_path_str}{i}.txt")
    checklistrecipe_string = ""
    with open(file_path, 'r') as f:
        text = f.read()
        text = separate_punct(text.lower().replace('\n', '')).strip()
        checklist_recipe_strings.append(text)

In [ ]:
checklist_df = pd.DataFrame(checklist_recipe_strings)
checklist_df.columns = ['text']
checklist_df['label'] = 3

In [ ]:
checklist_df['text'][0]

'🍴 curry goat   🥑  • 1 25 ounce package active dry yeast  • 1 8 ounce package cream cheese softened  • 1 2 cup unsalted butter softened  • 1 cup milk  • 1 cup milk  • 1 cup apple juice  • 1 teaspoon vanilla extract   🥣  ‣ preheat oven to 350 degrees  .   ‣ beat together butter in the flour  ,  baking powder  ,  baking soda water  .   ‣ coat the flour mixture out dry ingredients set  .   ‣ beat butter in a large saucepan  .  add the butter and egg yolk  .  add the flour mixture and continue until combined  .  add flour mixture and stir until combined  .  mix well and gently fold the mixture between 2 layers  .  bake until brioche cakes is golden brown  .   ‣ on the oven temperature to 350 degrees f  .  bake at least 5 minutes and set aside for 35 minutes  .  cool completely  ,  then carefully remove to make a dampening long pieces  .   ‣ sift together the flour and powdered sugar and butter into flour until they are combined  .  fold the bananas into food dish  .   ‣ roll up the dough a

In [ ]:
truth_df = pd.DataFrame(recipe_strings)
truth_df.columns = ['text']
truth_df['label'] = 0

In [ ]:
truth_df

,text,label
0,🍴 slow cooker chicken and dumplings 🥑 • 4 skin...,0
1,🍴 awesome slow cooker pot roast 🥑 • 2 ( 10 . ...,0
2,🍴 brown sugar meatloaf 🥑 • 1/2 cup packed brow...,0
3,🍴 best chocolate chip cookies 🥑 • 1 cup butter...,0
4,🍴 homemade mac and cheese casserole 🥑 • 8 ounc...,0
...,...,...
20161,🍴 georgia ' s tennessee jam cake 🥑 • 1 cup but...,0
20162,🍴 poached eggs and asparagus 🥑 • 4 eggs• 1 cub...,0
20163,🍴 bistecca alla fiorentina ( tuscan porterhou...,0
20164,🍴 courtney ' s three tomato pasta sauce 🥑 • 1/...,0


In [ ]:
truth_df

,text,label
0,🍴 slow cooker chicken and dumplings 🥑 • 4 skin...,0
1,🍴 awesome slow cooker pot roast 🥑 • 2 ( 10 . ...,0
2,🍴 brown sugar meatloaf 🥑 • 1/2 cup packed brow...,0
3,🍴 best chocolate chip cookies 🥑 • 1 cup butter...,0
4,🍴 homemade mac and cheese casserole 🥑 • 8 ounc...,0
...,...,...
20161,🍴 georgia ' s tennessee jam cake 🥑 • 1 cup but...,0
20162,🍴 poached eggs and asparagus 🥑 • 4 eggs• 1 cub...,0
20163,🍴 bistecca alla fiorentina ( tuscan porterhou...,0
20164,🍴 courtney ' s three tomato pasta sauce 🥑 • 1/...,0


In [ ]:
df = pd.concat([truth_df.sample(n=50), gruchar_df, gruword_df, checklist_df, gpt2_df])

In [ ]:
df

,text,label
16397,🍴 noodle pudding 🥑 • 1 ( 16 ounce ) package ...,0
20162,🍴 poached eggs and asparagus 🥑 • 4 eggs• 1 cub...,0
12075,🍴 reuben crescent bake 🥑 • 3/4 cup sauerkraut ...,0
18186,🍴 slow cooker apple butter with honey 🥑 • 5 1/...,0
3187,🍴 gnocchi bake 🥑 • 1 ( 18 ounce ) package fr...,0
...,...,...
45,🍴 cake mix cookies vii 🥑 • 2 cups white sugar•...,4
46,🍴 cheeseburger soup i 🥑 • 2 tablespoons olive ...,4
47,🍴 jalapeno buttered cornmeal bread 🥑 • 1 1/2 c...,4
48,🍴 orange dream bars 🥑 • 3/4 cup butter• 1/2 cu...,4


In [ ]:
df_path = os.path.join(CACHE_DIR, 'discrim_data_20210603.pkl')
df.to_pickle(df_path)

In [ ]:
df_path = os.path.join(CACHE_DIR, 'discrim_data_20210603.pkl')
if not os.path.exists(df_path):
    raise SystemExit("Run the above code before continuing.")
else:
    df_test = pd.read_pickle(df_path)

# Start here if you did the thing before

In [ ]:
df_test

,text,label
16397,🍴 noodle pudding 🥑 • 1 ( 16 ounce ) package ...,0
20162,🍴 poached eggs and asparagus 🥑 • 4 eggs• 1 cub...,0
12075,🍴 reuben crescent bake 🥑 • 3/4 cup sauerkraut ...,0
18186,🍴 slow cooker apple butter with honey 🥑 • 5 1/...,0
3187,🍴 gnocchi bake 🥑 • 1 ( 18 ounce ) package fr...,0
...,...,...
45,🍴 cake mix cookies vii 🥑 • 2 cups white sugar•...,4
46,🍴 cheeseburger soup i 🥑 • 2 tablespoons olive ...,4
47,🍴 jalapeno buttered cornmeal bread 🥑 • 1 1/2 c...,4
48,🍴 orange dream bars 🥑 • 3/4 cup butter• 1/2 cu...,4


#Import GPT-2 recipes and RNN recipes

GPT-2 recipes

In [ ]:
CACHE_DIR = "./drive/Shared drives/Capstone/tmp"
#pathlib.Path(CACHE_DIR).mkdir(exist_ok=True)
gpt2_recipes_path = os.path.join(CACHE_DIR, 'gpt2_105791_title_prompt_output_recipes')

In [ ]:
gpt2_recipe_strings = []

for i in tqdm(range(500)):
  file_path = os.path.join(gpt2_recipes_path, f"gpt2_recipe_{i}.txt")
  gpt2_recipe_string = ""
  with open(file_path, 'r') as f:
    gpt2_recipe_strings.append(f.read())


100%|██████████| 500/500 [04:16<00:00,  1.95it/s]


In [ ]:
len(gpt2_recipe_strings)

500

In [ ]:
gpt2_recipe_strings_df = pd.DataFrame(gpt2_recipe_strings)

In [ ]:
gpt2_recipe_strings_df

,0
0,Slow Cooker Turkey and Potatoes<ING>• 2 (16 ou...
1,Endive & Gorgonzola Salad <ING>• 3 tablespoons...
2,Curry-Spiced Minestrone<ING>• 2 tablespoons ol...
3,Tuna Dip II<ING>• 2 (5 ounce) cans solid white...
4,Pancetta Wrapped Bayou La Batre Shrimp with Ri...
...,...
495,Gina's Oatmeal Cookie Ice Cream Sandwiches<ING...
496,Chocolate Crown Pound Cake<ING>• 2 cups cake f...
497,Insalata di Puntarelle Livre<ING>• 1 1/2 poun...
498,Grandma's Ground Beef Casserole<ING>• 2 (16 ou...


In [ ]:
print(gpt2_recipe_strings[0])

Slow Cooker Turkey and Potatoes<ING>• 2 (16 ounce) packages frozen hash brown potatoes, thawed• 1 pound skinless, boneless turkey breast, cut into 1 inch pieces• 1 (10.75 ounce) can condensed cream of chicken soup• 1 cup milk• 1 cup shredded Cheddar cheese<INS>‣ In a slow cooker, combine the hash brown potatoes, turkey, soup, milk and cheese.‣ Cover and cook on Low for 8 to 10 hours, or until potatoes are tender.



Char-level RNN recipes

In [ ]:
#CACHE_DIR = "./drive/Shared drives/Capstone/tmp"
#pathlib.Path(CACHE_DIR).mkdir(exist_ok=True)
rnn_recipes_path = os.path.join(CACHE_DIR, 'rnn_title_prompt_output_recipes')

rnn_recipe_strings = []

for i in range(500):
  print(f"> {i} out of 500")
  file_path = os.path.join(rnn_recipes_path, f"rnn_recipe_{i}.txt")
  rnn_recipe_string = ""
  for line in open(file_path, 'r'):
    rnn_recipe_string += line
  rnn_recipe_strings.append(rnn_recipe_string)

> 0 out of 500
> 1 out of 500
> 2 out of 500
> 3 out of 500
> 4 out of 500
> 5 out of 500
> 6 out of 500
> 7 out of 500
> 8 out of 500
> 9 out of 500
> 10 out of 500
> 11 out of 500
> 12 out of 500
> 13 out of 500
> 14 out of 500
> 15 out of 500
> 16 out of 500
> 17 out of 500
> 18 out of 500
> 19 out of 500
> 20 out of 500
> 21 out of 500
> 22 out of 500
> 23 out of 500
> 24 out of 500
> 25 out of 500
> 26 out of 500
> 27 out of 500
> 28 out of 500
> 29 out of 500
> 30 out of 500
> 31 out of 500
> 32 out of 500
> 33 out of 500
> 34 out of 500
> 35 out of 500
> 36 out of 500
> 37 out of 500
> 38 out of 500
> 39 out of 500
> 40 out of 500
> 41 out of 500
> 42 out of 500
> 43 out of 500
> 44 out of 500
> 45 out of 500
> 46 out of 500
> 47 out of 500
> 48 out of 500
> 49 out of 500
> 50 out of 500
> 51 out of 500
> 52 out of 500
> 53 out of 500
> 54 out of 500
> 55 out of 500
> 56 out of 500
> 57 out of 500
> 58 out of 500
> 59 out of 500
> 60 out of 500
> 61 out of 500
> 62 out of 500
> 

In [ ]:
rnn_recipe_strings

['Cava Cocktail Sauce, diced into Matchstick strips\n\n\U0001f963\n‣ Preheat oven to 350 degrees F (175 degrees C). Grease a 9x13 inch baking pan.\n‣ In a small bowl, whisk together egg, milk, melted butter, vanilla extract and salt. Pour 1/4 cup souffle mixture into portobely and sprinkle with pecans. Pour batter into crust. Spread cream of cheese over ice. Mix well. Sprinkle dressing mixture over cake.',
 "Phil's Simple Butterscotch Apple Cider Mexican Pillsby's Pecan Chicken III\n\n\U0001f963\n‣ Preheat oven to 350 degrees F (175 degrees C).\n‣ Place bacon in a large heavy skillet brieg brown sugar together until soft, about 3 minutes. Add chicken and sauce. Simmer for 3 to 5 minutes or until steaks. Drain and cool completely.\n‣ Preheat oven to 400 degrees F (250 degrees C).\n‣ Bake cream cheese over medium heat until tender and mixture is evenly distributed.\n‣ In small bowl, mix together orzo, cloves, and Parmesan cheese.\n‣ Season with salt and pepper. Place chicken in pot with 

In [ ]:
rnn_recipe_strings[7][20:30]

'\n\U0001f963\n‣ In a '

In [ ]:
len(rnn_recipe_strings)

500

In [ ]:
# Clean up RNN recipe strings

cleaned_rnn_recipe_strings = []

for rnn_recipe_string in rnn_recipe_strings:
  cleaned_string = ""
  for char in rnn_recipe_string:
    if char == "\n":
      pass
    elif char == "🥑":
      cleaned_string += "<ING>"
    elif char == "\U0001f963":
      cleaned_string += "<INS>"
    else:
      cleaned_string += char
  cleaned_rnn_recipe_strings.append(cleaned_string)

In [ ]:
len(cleaned_rnn_recipe_strings)
cleaned_rnn_recipe_strings

['Cava Cocktail Sauce, diced into Matchstick strips<INS>‣ Preheat oven to 350 degrees F (175 degrees C). Grease a 9x13 inch baking pan.‣ In a small bowl, whisk together egg, milk, melted butter, vanilla extract and salt. Pour 1/4 cup souffle mixture into portobely and sprinkle with pecans. Pour batter into crust. Spread cream of cheese over ice. Mix well. Sprinkle dressing mixture over cake.',
 "Phil's Simple Butterscotch Apple Cider Mexican Pillsby's Pecan Chicken III<INS>‣ Preheat oven to 350 degrees F (175 degrees C).‣ Place bacon in a large heavy skillet brieg brown sugar together until soft, about 3 minutes. Add chicken and sauce. Simmer for 3 to 5 minutes or until steaks. Drain and cool completely.‣ Preheat oven to 400 degrees F (250 degrees C).‣ Bake cream cheese over medium heat until tender and mixture is evenly distributed.‣ In small bowl, mix together orzo, cloves, and Parmesan cheese.‣ Season with salt and pepper. Place chicken in pot with aluminum foil; spoon sauce over pi

In [ ]:
rnn_recipe_strings_df = pd.DataFrame(rnn_recipe_strings)

In [ ]:
rnn_recipe_strings_df

,0
0,"Cava Cocktail Sauce, diced into Matchstick str..."
1,Phil's Simple Butterscotch Apple Cider Mexican...
2,Lemon Lentils and Noodles used with any remove...
3,"Rosemary Simple Syrup, washed, to taste all of..."
4,"Austin, Texas Barbecue Paste III\n• 1 3/4 cups..."
...,...
495,Blueberries in Gin Syrup and Sauce with Chedda...
496,"Spicy Salsa, all a spring only Embertel chicke..."
497,Lemon Meringue Martini Chocolate Cookies Cooki...
498,Wrapped Asparagus are to pass halfway through ...


Save these datasets

In [ ]:
# dataset_path = os.path.join(CACHE_DIR, 'gpt2_title_output_recipes.pkl')
# gpt2_recipe_strings_df.to_pickle(dataset_path) 

In [ ]:
# dataset_path = os.path.join(CACHE_DIR, 'rnn_title_output_recipes.pkl')
# rnn_recipe_strings_df.to_pickle(dataset_path) 

#Training

Load in datasets

In [ ]:
gpt2_recipe_strings_df = pd.read_pickle(os.path.join(CACHE_DIR, "gpt2_title_output_recipes.pkl"))
rnn_recipe_strings_df = pd.read_pickle(os.path.join(CACHE_DIR, "rnn_title_output_recipes.pkl"))

Training-Test split

In [ ]:
# recipe_strings['label'] = 0 # ground truth

recipe_strings_df = pd.DataFrame(recipe_strings)
# recipe_strings_df.rename(columns={"0" : "text"})
recipe_strings_df['text'] = recipe_strings_df[0]
recipe_strings_df['label'] = 0 # ground truth
gpt2_recipe_strings_df['text'] = gpt2_recipe_strings_df[0]
gpt2_recipe_strings_df['label'] = 1 # GPT-2 finetuned
rnn_recipe_strings_df['text'] = rnn_recipe_strings_df[0]
rnn_recipe_strings_df['label'] = 2 # RNN

In [ ]:
recipe_strings_df['label']

0        0
1        0
2        0
3        0
4        0
        ..
20161    0
20162    0
20163    0
20164    0
20165    0
Name: label, Length: 20000, dtype: int64

In [ ]:
gpt2_recipe_strings_df

,0,text,label
0,Mango Lassi II<ING>• 2 cups frozen mango - pee...,Mango Lassi II<ING>• 2 cups frozen mango - pee...,1
1,Too Much Strawberry Cake<ING>• 2 cups cake flo...,Too Much Strawberry Cake<ING>• 2 cups cake flo...,1
2,Peach Cake I<ING>• 2 cups white sugar• 1 cup b...,Peach Cake I<ING>• 2 cups white sugar• 1 cup b...,1
3,Pineapple Orange Sorbet<ING>• 2 cups orange ju...,Pineapple Orange Sorbet<ING>• 2 cups orange ju...,1
4,Classy Green Bean Casserole<ING>• 2 (15 ounce)...,Classy Green Bean Casserole<ING>• 2 (15 ounce)...,1
...,...,...,...
495,Dijon-Tarragon Cream Chicken<ING>• 2 tablespoo...,Dijon-Tarragon Cream Chicken<ING>• 2 tablespoo...,1
496,Spicy Glazed Carrots<ING>• 2 cups uncooked bro...,Spicy Glazed Carrots<ING>• 2 cups uncooked bro...,1
497,Buffalo Chicken Pasta Salad<ING>• 2 (10 ounce)...,Buffalo Chicken Pasta Salad<ING>• 2 (10 ounce)...,1
498,Honey-Balsamic Chicken<ING>• 2 tablespoons oli...,Honey-Balsamic Chicken<ING>• 2 tablespoons oli...,1


In [ ]:
df.loc[df['label'].isin([0, 4])]

,text,label
16397,🍴 noodle pudding 🥑 • 1 ( 16 ounce ) package ...,0
20162,🍴 poached eggs and asparagus 🥑 • 4 eggs• 1 cub...,0
12075,🍴 reuben crescent bake 🥑 • 3/4 cup sauerkraut ...,0
18186,🍴 slow cooker apple butter with honey 🥑 • 5 1/...,0
3187,🍴 gnocchi bake 🥑 • 1 ( 18 ounce ) package fr...,0
...,...,...
45,🍴 cake mix cookies vii 🥑 • 2 cups white sugar•...,4
46,🍴 cheeseburger soup i 🥑 • 2 tablespoons olive ...,4
47,🍴 jalapeno buttered cornmeal bread 🥑 • 1 1/2 c...,4
48,🍴 orange dream bars 🥑 • 3/4 cup butter• 1/2 cu...,4


##Distinguishing GPT-2 from the ground truth

In [ ]:
truth_vs_gpt2_df = df.loc[df['label'].isin([0, 1])]

In [ ]:
truth_vs_gpt2_df

,text,label
16397,🍴 noodle pudding 🥑 • 1 ( 16 ounce ) package ...,0
20162,🍴 poached eggs and asparagus 🥑 • 4 eggs• 1 cub...,0
12075,🍴 reuben crescent bake 🥑 • 3/4 cup sauerkraut ...,0
18186,🍴 slow cooker apple butter with honey 🥑 • 5 1/...,0
3187,🍴 gnocchi bake 🥑 • 1 ( 18 ounce ) package fr...,0
...,...,...
45,"🍴 fettuccine with preserved tuna , capers , ...",1
46,🍴 apricot lentil soup ) ‣ follow marshmallow s...,1
47,🍴 roasted summer ratatouille ) . cook 1 onio...,1
48,🍴 melt-in-your-mouth braised and barbecued chi...,1


In [ ]:
truth_vs_gpt2_df['nwords'] = truth_vs_gpt2_df['text'].apply(lambda x: len(x.split()))

# truth_vs_gpt2_df = truth_vs_gpt2_df[truth_vs_gpt2_df['nwords']<350]

truth_vs_gpt2_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,text,label,nwords
16397,🍴 noodle pudding 🥑 • 1 ( 16 ounce ) package ...,0,231
20162,🍴 poached eggs and asparagus 🥑 • 4 eggs• 1 cub...,0,181
12075,🍴 reuben crescent bake 🥑 • 3/4 cup sauerkraut ...,0,298
18186,🍴 slow cooker apple butter with honey 🥑 • 5 1/...,0,91
3187,🍴 gnocchi bake 🥑 • 1 ( 18 ounce ) package fr...,0,159
...,...,...,...
45,"🍴 fettuccine with preserved tuna , capers , ...",1,211
46,🍴 apricot lentil soup ) ‣ follow marshmallow s...,1,56
47,🍴 roasted summer ratatouille ) . cook 1 onio...,1,62
48,🍴 melt-in-your-mouth braised and barbecued chi...,1,133


In [ ]:
truth_vs_gpt2_df['nwords'].max()

388

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping

In [ ]:
truth_vs_gpt2_df.drop(['nwords'], axis=1, inplace=True)
truth_vs_gpt2_df

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,text,label
16397,🍴 noodle pudding 🥑 • 1 ( 16 ounce ) package ...,0
20162,🍴 poached eggs and asparagus 🥑 • 4 eggs• 1 cub...,0
12075,🍴 reuben crescent bake 🥑 • 3/4 cup sauerkraut ...,0
18186,🍴 slow cooker apple butter with honey 🥑 • 5 1/...,0
3187,🍴 gnocchi bake 🥑 • 1 ( 18 ounce ) package fr...,0
...,...,...
45,"🍴 fettuccine with preserved tuna , capers , ...",1
46,🍴 apricot lentil soup ) ‣ follow marshmallow s...,1
47,🍴 roasted summer ratatouille ) . cook 1 onio...,1
48,🍴 melt-in-your-mouth braised and barbecued chi...,1


In [ ]:
X = truth_vs_gpt2_df.text
Y = truth_vs_gpt2_df.label

In [ ]:
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1, 1)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.5)

In [ ]:
max_words = 1000
max_len = 150
tokenizer = Tokenizer(num_words = max_words)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences, maxlen = max_len)

In [ ]:
sequences_matrix

array([[  0,   0,   0, ..., 102,   2, 379],
       [646, 380,  28, ..., 224, 225, 651],
       [  1,   4,   5, ..., 386,  48, 315],
       ...,
       [596,  95, 108, ..., 340,  48, 315],
       [ 10,   5, 110, ...,  39,  99,  14],
       [ 73,  55,  23, ..., 144,   2, 137]], dtype=int32)

In [ ]:
def discriminator_RNN():
  inputs = Input(name='inputs', shape=[max_len])
  layer = Embedding(max_words, 100, input_length=max_len)(inputs)
  layer = LSTM(128)(layer)
  layer = Dense(512, name='FC1')(layer)
  layer = Activation('relu')(layer)
  layer = Dropout(0.5)(layer)
  layer = Dense(1, name='out_layer')(layer)
  layer = Activation('sigmoid')(layer)
  model = Model(inputs=inputs, outputs=layer)
  return model

In [ ]:
model = discriminator_RNN()
model.summary()
model.compile(loss='binary_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 150)]             0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 150, 100)          100000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               117248    
_________________________________________________________________
FC1 (Dense)                  (None, 512)               66048     
_________________________________________________________________
activation_4 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 513 

In [ ]:
model.fit(sequences_matrix, Y_train, batch_size=10, epochs=10,
          validation_split=0.2, callbacks=[]) # EarlyStopping(monitor='val_loss', min_delta=0.0)])

Epoch 1/10
4/4 [==============================] - 0s 23ms/step - loss: 0.0074 - accuracy: 1.0000 - val_loss: 0.8057 - val_accuracy: 0.6000
Epoch 2/10
4/4 [==============================] - 0s 15ms/step - loss: 0.0045 - accuracy: 1.0000 - val_loss: 0.8770 - val_accuracy: 0.5000
Epoch 3/10
4/4 [==============================] - 0s 17ms/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 0.9603 - val_accuracy: 0.5000
Epoch 4/10
4/4 [==============================] - 0s 16ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 1.0237 - val_accuracy: 0.5000
Epoch 5/10
4/4 [==============================] - 0s 15ms/step - loss: 0.0010 - accuracy: 1.0000 - val_loss: 1.1250 - val_accuracy: 0.5000
Epoch 6/10
4/4 [==============================] - 0s 16ms/step - loss: 8.1281e-04 - accuracy: 1.0000 - val_loss: 1.2389 - val_accuracy: 0.5000
Epoch 7/10
4/4 [==============================] - 0s 15ms/step - loss: 5.0663e-04 - accuracy: 1.0000 - val_loss: 1.2681 - val_accuracy: 0.5000
Epoch 8/10
4/4 [===

In [ ]:
test_sequences = tokenizer.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences, maxlen = max_len)

In [ ]:
accuracy = model.evaluate(test_sequences_matrix, Y_test)
print('Test set\nLoss: {:0.3f}\n Accuracy {:0.3f}'.format(accuracy[0], accuracy[1]))

2/2 [==============================] - 0s 9ms/step - loss: 1.9399 - accuracy: 0.5000
Test set
Loss: 1.940
 Accuracy 0.500


##Distinguishing the RNN from the ground truth

In [ ]:
truth_vs_rnn_df = pd.concat([recipe_strings_df.sample(n=500), rnn_recipe_strings_df.sample(n=500)])

truth_vs_rnn_df

,0,text,label
1761,Chicken and Broccoli Alfredo<ING>• 1/2 (16 oun...,Chicken and Broccoli Alfredo<ING>• 1/2 (16 oun...,0
2444,Sweet Potato Pie II<ING>• 2 cups mashed sweet ...,Sweet Potato Pie II<ING>• 2 cups mashed sweet ...,0
5180,World's Best Bacon Cheese Dip<ING>• 4 slices b...,World's Best Bacon Cheese Dip<ING>• 4 slices b...,0
15855,Rhubarb Upside Down Cake III<ING>• 5 cups chop...,Rhubarb Upside Down Cake III<ING>• 5 cups chop...,0
8833,Mom's Chili<ING>• 1 pound ground beef\n• 1 lar...,Mom's Chili<ING>• 1 pound ground beef\n• 1 lar...,0
...,...,...,...
414,Raisin Lemon Cookies III '0 we from lingossed ...,Raisin Lemon Cookies III '0 we from lingossed ...,2
431,Baked Potato Croquettes until crumbled and res...,Baked Potato Croquettes until crumbled and res...,2
332,Vietnamese-Style Sandwich: Banh Mixed Brownies...,Vietnamese-Style Sandwich: Banh Mixed Brownies...,2
210,"Strawberry, Spinach, and Pear Salad dressing, ...","Strawberry, Spinach, and Pear Salad dressing, ...",2


In [ ]:
truth_vs_rnn_df['nwords'] = truth_vs_rnn_df['text'].apply(lambda x: len(x.split()))

truth_vs_rnn_df = truth_vs_rnn_df[truth_vs_rnn_df['nwords']<350]

truth_vs_rnn_df

,0,text,label,nwords
1761,Chicken and Broccoli Alfredo<ING>• 1/2 (16 oun...,Chicken and Broccoli Alfredo<ING>• 1/2 (16 oun...,0,112
2444,Sweet Potato Pie II<ING>• 2 cups mashed sweet ...,Sweet Potato Pie II<ING>• 2 cups mashed sweet ...,0,146
5180,World's Best Bacon Cheese Dip<ING>• 4 slices b...,World's Best Bacon Cheese Dip<ING>• 4 slices b...,0,108
15855,Rhubarb Upside Down Cake III<ING>• 5 cups chop...,Rhubarb Upside Down Cake III<ING>• 5 cups chop...,0,141
8833,Mom's Chili<ING>• 1 pound ground beef\n• 1 lar...,Mom's Chili<ING>• 1 pound ground beef\n• 1 lar...,0,93
...,...,...,...,...
414,Raisin Lemon Cookies III '0 we from lingossed ...,Raisin Lemon Cookies III '0 we from lingossed ...,2,155
431,Baked Potato Croquettes until crumbled and res...,Baked Potato Croquettes until crumbled and res...,2,63
332,Vietnamese-Style Sandwich: Banh Mixed Brownies...,Vietnamese-Style Sandwich: Banh Mixed Brownies...,2,128
210,"Strawberry, Spinach, and Pear Salad dressing, ...","Strawberry, Spinach, and Pear Salad dressing, ...",2,192


In [ ]:
truth_vs_rnn_df['nwords'].max()

349

In [ ]:
truth_vs_rnn_df.drop([0, 'nwords'], axis=1, inplace=True)
truth_vs_rnn_df

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,text,label
1761,Chicken and Broccoli Alfredo<ING>• 1/2 (16 oun...,0
2444,Sweet Potato Pie II<ING>• 2 cups mashed sweet ...,0
5180,World's Best Bacon Cheese Dip<ING>• 4 slices b...,0
15855,Rhubarb Upside Down Cake III<ING>• 5 cups chop...,0
8833,Mom's Chili<ING>• 1 pound ground beef\n• 1 lar...,0
...,...,...
414,Raisin Lemon Cookies III '0 we from lingossed ...,2
431,Baked Potato Croquettes until crumbled and res...,2
332,Vietnamese-Style Sandwich: Banh Mixed Brownies...,2
210,"Strawberry, Spinach, and Pear Salad dressing, ...",2


In [ ]:
X = truth_vs_rnn_df.text
Y = truth_vs_rnn_df.label

In [ ]:
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1, 1)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

In [ ]:
max_words = 1000
max_len = 150
tokenizer = Tokenizer(num_words = max_words)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences, maxlen = max_len)

In [ ]:
sequences_matrix

array([[  0,   0,   0, ...,  17, 132, 181],
       [ 16,   1,   2, ..., 431, 132, 181],
       [  0,   0,   0, ...,  19,  73,  17],
       ...,
       [  0,   0,   0, ...,  17, 132, 445],
       [245,  60,   1, ...,  32, 542, 517],
       [122, 104,   1, ...,   9, 300,  17]], dtype=int32)

In [ ]:
model = discriminator_RNN()
model.summary()
model.compile(loss='binary_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 150)]             0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 150, 100)          100000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               117248    
_________________________________________________________________
FC1 (Dense)                  (None, 512)               66048     
_________________________________________________________________
activation_2 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 513 

In [ ]:
model.fit(sequences_matrix, Y_train, batch_size=128, epochs=10,
          validation_split=0.2, callbacks=[EarlyStopping(monitor='val_loss', min_delta=0.0001)])

Epoch 1/10
5/5 [==============================] - 0s 49ms/step - loss: 0.1805 - accuracy: 0.9451 - val_loss: 0.9059 - val_accuracy: 0.6438
Epoch 2/10
5/5 [==============================] - 0s 34ms/step - loss: 0.2448 - accuracy: 0.9089 - val_loss: 0.5740 - val_accuracy: 0.7312
Epoch 3/10
5/5 [==============================] - 0s 30ms/step - loss: 0.1009 - accuracy: 0.9765 - val_loss: 0.5736 - val_accuracy: 0.7563
Epoch 4/10
5/5 [==============================] - 0s 29ms/step - loss: 0.1428 - accuracy: 0.9372 - val_loss: 0.6438 - val_accuracy: 0.6938


In [ ]:
test_sequences = tokenizer.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences, maxlen = max_len)

In [ ]:
accuracy = model.evaluate(test_sequences_matrix, Y_test)
print('Test set\nLoss: {:0.3f}\n Accuracy {:0.3f}'.format(accuracy[0], accuracy[1]))

7/7 [==============================] - 0s 7ms/step - loss: 0.7928 - accuracy: 0.6900
Test set
Loss: 0.793
 Accuracy 0.690


# Leftover DistilBERT stuff (ignore)

In [ ]:
import ppb

ModuleNotFoundError: ignored

In [ ]:
model_class, tokenizer_class, pretrained_weights = ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased'

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

NameError: ignored

In [ ]:
tokenized = truth_vs_rnn_df['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

tokenized

15677    [101, 4086, 1011, 1050, 1011, 14768, 20963, 22...
15670    [101, 2665, 14068, 1998, 2004, 28689, 12349, 1...
858      [101, 4086, 1998, 14768, 15960, 3523, 1026, 13...
655      [101, 7975, 1998, 20377, 28168, 16220, 10624, ...
18224    [101, 12183, 3527, 1026, 13749, 1028, 1528, 10...
                               ...                        
51       [101, 2175, 18581, 25650, 2721, 1024, 6366, 24...
30       [101, 1040, 1005, 6253, 5044, 8808, 2452, 1795...
238      [101, 25935, 1011, 21229, 11345, 100, 1528, 10...
432      [101, 16510, 8091, 1005, 1055, 6904, 19570, 20...
32       [101, 2531, 1003, 7427, 4487, 16643, 11001, 23...
Name: text, Length: 197, dtype: object

In [ ]:
max_len = max([len(i) for i in tokenized.values])

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(197, 469)

In [ ]:
input_ids = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
  last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()

In [ ]:
features.shape

(197, 768)

In [ ]:
labels = truth_vs_rnn_df['label']
labels

15677    0
15670    0
858      0
655      0
18224    0
        ..
51       2
30       2
238      2
432      2
32       2
Name: label, Length: 197, dtype: int64

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [ ]:
class FFNet(nn.Module):
  def __init__(self, input_dim=768, hidden_dim=1024, output_dim=1, dropout=0.8):
    super(FFNet, self).__init__()
    self.fc1 = nn.Sequential(
        nn.Linear(input_dim, hidden_dim),
        nn.Dropout(dropout),
        nn.LeakyReLU(),
        nn.BatchNorm1d(hidden_dim),
    )
    self.fc3 = nn.Sequential(
        nn.Linear(hidden_dim, output_dim),
        nn.Sigmoid(),
    )

  def forward(self, x):
    x = self.fc1(x)
    x = self.fc3(x)
    return x

ffnet = FFNet()

In [ ]:
criterion = nn.BCELoss()
optimizer = optim.SGD(ffnet.parameters(), lr=0.001, momentum=0.9)

In [ ]:
class Dataset(torch.utils.data.Dataset):
  def __init__(self, features, labels):
    self.features = features
    self.labels = labels
  def __len__(self):
    return self.features.shape[0]
  def __getitem__(self, index):
    X = self.features[index,:]
    y = self.labels[index]
    return X, y

train_dataset = Dataset(train_features, train_labels.to_numpy())

In [ ]:
trainloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True
)

In [ ]:
NUM_EPOCHS = 1000

for epoch in range(NUM_EPOCHS):
  running_loss = 0.0
  for data, labels in trainloader:
    optimizer.zero_grad()
    outputs = ffnet(data)
    loss = criterion(outputs, labels.float().unsqueeze(1))
    loss.backward()
    optimizer.step()
    #print(outputs.view(1, -1))
    #print(labels.view(1, -1))

    running_loss += loss.item()
  if epoch % 50 == 0:
    print('Epoch {}, loss: {}'.format(epoch, running_loss))

print('Finished training')

Epoch 0, loss: 11.31788244843483
Epoch 50, loss: -194.97640949487686
Epoch 100, loss: -242.69247835874557
Epoch 150, loss: -200.0297458767891
Epoch 200, loss: -229.0006217956543
Epoch 250, loss: -311.5216683149338
Epoch 300, loss: -461.4820215702057
Epoch 350, loss: -306.5561623573303
Epoch 400, loss: -310.3633278235793
Epoch 450, loss: -332.66350173950195
Epoch 500, loss: -300.150194644928
Epoch 550, loss: -188.09028300642967
Epoch 600, loss: -300.6163331270218
Epoch 650, loss: -436.1982421875
Epoch 700, loss: -252.1306470632553
Epoch 750, loss: -274.73557567596436
Epoch 800, loss: -217.45036166906357
Epoch 850, loss: -244.4103483557701
Epoch 900, loss: -227.3741238117218
Epoch 950, loss: -151.6368461647071
Finished training


In [ ]:
test_dataset = Dataset(test_features, test_labels.to_numpy())

testloader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=True
)

In [ ]:
y_pred = []
y = []
#total = 0
#correct = 0
with torch.no_grad():
  for data, labels in testloader:
    outputs = ffnet(data)
    predicted = torch.LongTensor(np.where(outputs > 0.5, 2, 0)).view(-1)
    y_pred.extend(predicted.tolist())
    y.extend(labels.tolist())
    #total += labels.size(0)
    #correct += (predicted == labels).sum().item()

print(y_pred)
print(y)

print('Accuracy of the FFNet trained on BERT sentence embeddings\non the test sentences: %0.3f %%' % accuracy_score(np.array(y), np.array(y_pred)))
print('F1-score of the FFNet trained on BERT sentence embeddings\non the test sentences: %0.3f %%' % f1_score(np.array(y), np.array(y_pred), average='micro'))

[2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2]
[2, 2, 2, 0, 0, 2, 0, 0, 2, 2, 0, 0, 2, 2, 0, 2, 0, 0, 0, 2, 2, 2, 0, 2, 0, 2, 0, 0, 2, 2, 2, 0, 0, 2, 0, 0, 2, 0, 2, 0, 2, 2, 0, 2, 0, 0, 0, 0, 2, 0]
Accuracy of the FFNet trained on BERT sentence embeddings
on the test sentences: 0.600 %
F1-score of the FFNet trained on BERT sentence embeddings
on the test sentences: 0.600 %
